# Progetto BigData
## Valerio Di Zio e Francesco Magnani

intro here

In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2324-vdizio"

val path_book_sample = "s3a://"+bucketname+"/project/book_sample134.json"
val path_book_sample32row = "s3a://"+bucketname+"/project/book_sample32row.json"
val path_positive_words = "s3a://"+bucketname+"/project/positive-words.txt"
val path_negative_words = "s3a://"+bucketname+"/project/negative-words.txt"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1707475848681_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-vdizio
path_book_sample: String = s3a://unibo-bd2324-vdizio/project/book_sample134.json
path_book_sample32row: String = s3a://unibo-bd2324-vdizio/project/book_sample32row.json
path_positive_words: String = s3a://unibo-bd2324-vdizio/project/positive-words.txt
path_negative_words: String = s3a://unibo-bd2324-vdizio/project/negative-words.txt
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1707475848681_0001/


In [3]:
case class BookReview(
    id: String,
    overall:Double,
    reviewTime:String,
    reviewerID:String,
    asin:String,
    reviewText:String,
    summary:String
)

object BookReview{
    def extract(row:org.apache.spark.sql.Row) = {
        val overall = row.getAs[Double]("overall")
        val reviewTime = row.getAs[String]("reviewTime")
        val reviewerID = row.getAs[String]("reviewerID")
        val asin = row.getAs[String]("asin")
        val reviewText = row.getAs[String]("reviewText")
        val summary = row.getAs[String]("summary")
        val id = reviewerID + "-" + asin
    
        new BookReview(id, overall, reviewTime, reviewerID, asin, reviewText, summary)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class BookReview
defined object BookReview
Companions must be defined together; you may wish to use :paste mode for this.


In [4]:
val bookSampleRdd = spark.read.json(path_book_sample).map(BookReview.extract)
val bookSampleRdd32row = spark.read.json(path_book_sample32row).map(BookReview.extract)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bookSampleRdd: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]
bookSampleRdd32row: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]


In [5]:
val positiveWords = sc.textFile(path_positive_words)
val negativeWords = sc.textFile(path_negative_words)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/positive-words.txt MapPartitionsRDD[9] at textFile at <console>:24
negativeWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/negative-words.txt MapPartitionsRDD[11] at textFile at <console>:24


In [13]:
def countSubstringOccurrences(text: String, substring: String): Int = {
  // Constructing the regex pattern to match the exact substring
  val pattern = s"(?<!\\S)${java.util.regex.Pattern.quote(substring)}(?!\\S)".r
  
  // Counting the occurrences of the substring using findAllMatchIn
  pattern.findAllMatchIn(text).size
}

def countPositive(text: String) = positiveWords.map(word => countSubstringOccurrences(text, word)).sum
def countNegative(text: String) = negativeWords.map(word => countSubstringOccurrences(text, word)).sum

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

countSubstringOccurrences: (text: String, substring: String)Int
countPositive: (text: String)Double
countNegative: (text: String)Double


In [14]:
countPositive("This is a cute story my children used to love. Now my grandson will get a chance to read it as well.  The book has been out of print so I was happy to find a copy.  It arrived promptly in near new condition as described in the sellers comments..")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res14: Double = 3.0


In [7]:
def splitAndClean(id:String, text: String): Array[String] = 
    text.split("\\s+").map(el => id + "_" + el.toLowerCase.replaceAll("[^a-zA-Z\\s]", ""))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitAndClean: (id: String, text: String)Array[String]


In [8]:
splitAndClean("acaso", "Ciao pesco cesco\n12h323232!!!!d")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res6: Array[String] = Array(acaso_ciao, acaso_pesco, acaso_cesco, acaso_hd)


In [55]:
val positiveDf = positiveWords.toDF("positiveWord")
val negativeDf = negativeWords.toDF("negativeWord")

val reviewsRdd = bookSampleRdd32row.
    map(review => (review.id, review.reviewText)).
    filter(review => review._2 != null && review._2.length > 3).
    flatMap({case (id, text) => splitAndClean(id, text)}).
    filter(text => text.length > 3).
    toDF("word")

val withPositiveRdd = reviewsRdd.
    join(positiveDf, expr("SUBSTRING(word, instr(word, '_')+1, LEN(word)) like positiveWord")).
    map(row => (row.getAs[String](0), row.getAs[String](1))).
    map({case (el, positiveWord) => (el.substring(0, el.indexOf('_')), 1)}).rdd.
    reduceByKey(_ + _)
    
val withNegativeRdd = reviewsRdd.
    join(negativeDf, expr("SUBSTRING(word, instr(word, '_')+1, LEN(word)) like negativeWord")).
    map(row => (row.getAs[String](0), row.getAs[String](1))).
    map({case (el, negativeWords) => (el.substring(0, el.indexOf('_')), 1)}).
    rdd.
    reduceByKey(_ + _)

val result = withPositiveRdd.
    fullOuterJoin(withNegativeRdd)

val reducedResult = result.
    map ({ case (key, (value1, value2)) => (key, (value1.getOrElse(0) - value2.getOrElse(0)))})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveDf: org.apache.spark.sql.DataFrame = [positiveWord: string]
negativeDf: org.apache.spark.sql.DataFrame = [negativeWord: string]
reviewsRdd: org.apache.spark.sql.DataFrame = [word: string]
withPositiveRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[1041] at reduceByKey at <console>:29
withNegativeRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[1058] at reduceByKey at <console>:30
result: org.apache.spark.rdd.RDD[(String, (Option[Int], Option[Int]))] = MapPartitionsRDD[1061] at fullOuterJoin at <console>:26
reducedResult: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[1062] at map at <console>:25


In [56]:
// Troviamo il valore minimo e massimo
val minValue = reducedResult.values.min()
val maxValue = reducedResult.values.max()

// Calcoliamo la lunghezza dell'intervallo
val intervalLength = (maxValue - minValue) / 5.0

// Creiamo i 5 intervalli e assegniamo loro un numero identificativo
val intervals = (0 until 5).map { i =>
  val start = minValue + intervalLength * i
  val end = minValue + intervalLength * (i + 1)
  (i + 1, start, end)
}

// Mappa ogni valore nel suo intervallo corrispondente
val result = reducedResult.mapValues { value =>
  val interval = intervals.find { case (_, start, end) =>
    value >= start && value < end
  }.getOrElse((-1, Double.NaN, Double.NaN))
  interval._1
}

result.foreach(println)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

minValue: Int = -6
maxValue: Int = 7
intervalLength: Double = 2.6
intervals: scala.collection.immutable.IndexedSeq[(Int, Double, Double)] = Vector((1,-6.0,-3.4), (2,-3.4,-0.7999999999999998), (3,-0.7999999999999998,1.8000000000000007), (4,1.8000000000000007,4.4), (5,4.4,7.0))
result: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[1065] at mapValues at <console>:26


In [15]:
bookSampleRdd32row.collect().foreach(f => println(f.id, countPositive(f.reviewText.toLowerCase.replaceAll("[^a-zA-Z\\s]", ""))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(A1C6M8LCIX4M6M-0001713353,3.0)
(A1REUF3A1YCPHM-0001713353,10.0)
(A1YRBRK2XM5D5-0001713353,6.0)
(A1V8ZR5P78P4ZU-0001713353,4.0)
(A2ZB06582NXCIV-0001713353,3.0)
(ACPQVNRD3Z09X-0001713353,3.0)
(AVP0HXC9FG790-0001713353,1.0)
(A32MQTLQQN44WW-0001713353,1.0)
(A13CHIJPFCEP2M-0001713353,1.0)
(A324TTUBKTN73A-0001713353,2.0)
(ADLBKEDYXKXCH-0001713353,2.0)
(A3MVI1KIZQA9XB-0001713353,5.0)
(A2RE7WG349NV5D-0001713353,1.0)
(A2GSWEYW6Y4OIY-0001713353,2.0)
(AX8YZYILXC7LT-0001713353,0.0)
(ABZ2UEQ2JRYJ9-0001713353,1.0)
(A32B7QIUDQCD0E-0001713353,1.0)
(A25PQPZXU63223-0001713353,1.0)
(AUZXLPH8BN28C-0001713353,1.0)
(A2VXMOSOTK0W70-0001713353,3.0)
(A232U4KSNYAO2R-0001713353,0.0)
(A1RDVO9NPKWISU-0001713353,2.0)
(AHHHGMN33KF95-0001713353,2.0)
(A3H9YD6K9TVKDP-0001713353,11.0)
(A3M314LI9OYME-0001713353,2.0)
(A1YDQQJDRHM0FJ-0001713353,3.0)
(A1IL1A91CHGU53-0001713353,2.0)
(AHXMDCU0N15TN-0001713353,4.0)
(A2M46WTE5TR5WN-0001713353,0.0)
(ATHTCOG6BB6WK-0001713353,1.0)
